<a href="https://colab.research.google.com/github/oushiro/aligner/blob/main/overlapremove_toshare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textgrid

In [ ]:
import os
import textgrid

# Carregar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminhos para as pastas de entrada e saída
input_dir = '/content/drive/MyDrive/overlap_remove'  # Pasta onde estão os arquivos TextGrid
output_dir = '/content/drive/MyDrive/processed_files'  # Pasta onde os arquivos processados serão salvos

# Se a pasta de saída não existir, crie-a
os.makedirs(output_dir, exist_ok=True)

# Obter todos os arquivos TextGrid
textgrid_files = [f for f in os.listdir(input_dir) if f.endswith('.TextGrid')]

# Verificar se há arquivos TextGrid
if len(textgrid_files) == 0:
    print("Nenhum arquivo TextGrid encontrado")
else:
    # Percorrer todos arquivos TextGrid
    for file_name in textgrid_files:
        input_file = os.path.join(input_dir, file_name)
        output_file = os.path.join(output_dir, file_name)  # Manter o mesmo nome para o arquivo de saída

        # Abrir o arquivo TextGrid
        tg = textgrid.TextGrid.fromFile(input_file)

        # Visualizar as camadas no TextGrid
        print(f"Processando o arquivo: {file_name}")
        print("As camadas neste arquivo TextGrid incluem:")
        for tier in tg.tiers:
            print(f"Nome da camada: {tier.name}")

        # Obter a camada-alvo e a camada de sobreposição
        target_tier_name = 'S1'  # Supondo que o nome da camada-alvo seja 'S1'
        overlap_tier_name = 'Overlap'  # Supondo que o nome da camada de sobreposição seja 'Overlap'

        target_tier = None
        overlap_tier = None

        # Localizar a camada-alvo e a camada de sobreposição
        for tier in tg.tiers:
            if tier.name == target_tier_name:
                target_tier = tier
            if tier.name == overlap_tier_name:
                overlap_tier = tier

        # Se as camadas forem encontradas, realizar a exclusão das anotações
        if target_tier and overlap_tier:
            target_intervals = target_tier.intervals
            overlap_intervals = overlap_tier.intervals

            # Percorrer todas anotações na camada-alvo
            for target in target_intervals:
                target_start = target.minTime  # Obter o tempo inicial da anotação na camada-alvo
                target_end = target.maxTime    # Obter o tempo final da anotação na camada-alvo

                # Percorrer todas anotações na camada de sobreposição
                for overlap in overlap_intervals:
                    overlap_start = overlap.minTime  # Obter o tempo inicial da anotação na camada de sobreposição
                    overlap_end = overlap.maxTime    # Obter o tempo final da anotação na camada de sobreposição

                    # Se a anotação-alvo e a anotação de sobreposição coincidirem, excluir a anotação-alvo
                    if overlap_start >= target_start and overlap_end <= target_end:
                        target.mark = ""  # Limpar o conteúdo dessa anotação
                        break  # Sobreposição confirmada, não é necessário continuar verificando

            # Salvar o arquivo TextGrid modificado
            tg.write(output_file)
            print(f"O arquivo TextGrid processado foi salvo em {output_file}")
        else:
            print("Camada-alvo ou camada de sobreposição não encontrada. Verifique os nomes das camadas.")